In [2]:
#Read Data

In [ ]:
# Library
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn import linear_model
cv2.__version__

In [ ]:
#General Settings
folder = 'video-data/kitti/2011_09_26_drive_0001_extract/image_02/data/'
image = '0000000100.png'

In [ ]:
def getImage(): 
    img = cv2.imread(folder + image) # reads image 'opencv-logo.png' as grayscale    plt.imshow(img)
    return img

In [ ]:
def getGrayImage(): 
    # Convert to Grayscale
    return cv2.cvtColor(getImage(), cv2.COLOR_BGR2GRAY) 

In [10]:
# Reshape Image
def reshapeImage():
    resized_image = cv2.resize(getGrayImage(), (750, 250)) 

In [17]:
def gaussianBlur():
    return cv2.GaussianBlur(getGrayImage(),(5,5),0)

In [60]:
def regionOfInterest(image):
    height = image.shape[0]
    triangle = np.array([[(0, height), (1000, height), (680, 200)]])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, triangle, 255)
    return cv2.bitwise_and(image, mask)
    #return mask

In [57]:
def regionOfInterestOverlay(image):
    height = image.shape[0]
    triangle = np.array([[(0, height), (1000, height), (680, 200)]])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, triangle, 255)
    return mask

In [14]:
def displayLines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 10)
    return line_image   

In [83]:
def undistortImage():
    cv2.undistort(getGrayImage())

In [98]:
def extract_lane(road_lines):
    left_lane = []
    right_lane = []
    left_slope = []
    right_slope = []

    if road_lines is not None:
        for x in range(0, len(road_lines)):
            for x1,y1,x2,y2 in road_lines[x]:
                slope = compute_slope(x1,y1,x2,y2)
                if (slope < 0):
                    left_lane.append(road_lines[x])
                    left_slope.append(slope)
                else:
                    if (slope > 0):
                        right_lane.append(road_lines[x])
                        right_slope.append(slope)
                
    return left_lane, right_lane , left_slope, right_slope

def compute_slope(x1,y1,x2,y2):
    if x2!=x1:
        return ((y2-y1)/(x2-x1))

In [107]:
def split_append(left_lane, right_lane):
    left_lane_sa = []
    right_lane_sa = []
    
    for x in range(0, len(left_lane)):
        for x1,y1,x2,y2 in left_lane[x]:
            left_lane_sa.append([x1, y1])
            left_lane_sa.append([x2, y2])

    for y in range(0, len(right_lane)):
        for x1,y1,x2,y2 in right_lane[y]:
            right_lane_sa.append([x1,y1])
            right_lane_sa.append([x2,y2])
            
    left_lane_sa = np.array(left_lane_sa)
    right_lane_sa = np.array(right_lane_sa)
    left_lane_sa,right_lane_sa = sort(left_lane_sa,right_lane_sa)
    return left_lane_sa,right_lane_sa

def sort(left_lane_sa,right_lane_sa):
    left_lane_sa = left_lane_sa[np.argsort(left_lane_sa[:, 0])]
    right_lane_sa = right_lane_sa[np.argsort(right_lane_sa[:, 0])]

    return left_lane_sa, right_lane_sa

In [108]:
def ransac_drawlane(left_lane_sa, right_lane_sa,frame):
    left_lane_x = []
    left_lane_y = []
    right_lane_x = []
    right_lane_y = []

    for x1,y1 in left_lane_sa:
        left_lane_x.append([x1])
        left_lane_y.append([y1])

    for x1,y1 in right_lane_sa:
        right_lane_x.append([x1])
        right_lane_y.append([y1])

    left_ransac_x = np.array(left_lane_x)
    left_ransac_y = np.array(left_lane_y)

    right_ransac_x = np.array(right_lane_x)
    right_ransac_y = np.array(right_lane_y)

        
    left_ransac = linear_model.RANSACRegressor(linear_model.LinearRegression())
    #print(left_ransac_x,left_ransac_y,len(left_ransac_x),len(left_ransac_y), left_ransac_x.shape )
    left_ransac.fit(left_ransac_x, left_ransac_y)
    slope_left = left_ransac.estimator_.coef_
    intercept_left = left_ransac.estimator_.intercept_

    right_ransac = linear_model.RANSACRegressor()
    right_ransac.fit(right_ransac_x, right_ransac_y)
    slope_right = right_ransac.estimator_.coef_
    intercept_right = right_ransac.estimator_.intercept_

    ysize = frame.shape[0]
    xsize = frame.shape[1]
    y_limit_low = int(0.95*ysize)
    y_limit_high = int(0.65*ysize)

    #Coordinates for point 1(Bottom Left)
    y_1 = ysize
    x_1 = int((y_1-intercept_left)/slope_left)

    #Coordinates for point 2(Bottom Left)
    y_2 = y_limit_high
    x_2 = int((y_2-intercept_left)/slope_left)

    #Coordinates for point 3(Bottom Left)
    y_3 = y_limit_high
    x_3 = int((y_3-intercept_right)/slope_right)
    
    #Coordinates for point 4(Bottom Right)
    y_4 = ysize
    x_4 = int((y_4-intercept_right)/slope_right)

    cv2.line(frame,(x_1,y_1),(x_2,y_2),(0,255,255),3)
    cv2.line(frame,(x_3,y_3),(x_4,y_4),(0,255,255),3)
    pts = np.array([[x_1, y_1], [x_2, y_2], [x_3, y_3], [x_4, y_4]])
    mask_color = (255,255,0)
    frame_copy = frame.copy()
    cv2.fillPoly(frame_copy, np.int32([pts]), mask_color)
    opacity = 0.4
    cv2.addWeighted(frame_copy,opacity,frame,1-opacity,0,frame)
    return frame
      

In [ ]:
# Step 1. Cut the ROI
roi_image = (regionOfInterest(getImage())) 
imgHLS = cv2.cvtColor(roi_image, cv2.COLOR_BGR2HLS)
canny_image = cv2.Canny(imgHLS,100,200)



lines = cv2.HoughLinesP(canny_image, 1, np.pi/180, 50, maxLineGap=50)
left_lane, right_lane, left_slope, right_slope = extract_lane(lines)
left_lane_sa, right_lane_sa = split_append(left_lane, right_lane)
image_np = ransac_drawlane(left_lane_sa, right_lane_sa,getImage())

plt.imshow(image_np)

In [ ]:
plt.imshow(getImage())